In [ ]:

!git clone --recursive https://github.com/Kacper-M-Michalik/splatter-image.git
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git

In [ ]:
%cd /content/splatter-image
!pip install -r requirements.txt

In [ ]:
# Get correct package versions
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
# !pip install gsplat

%cd /content/gaussian-splatting
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.0;7.5;8.0+PTX"
!pip install -e submodules/diff-gaussian-rasterization

In [ ]:
from google.colab import drive
import torch
import numpy as np
import matplotlib.pyplot as plt

print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)

In [ ]:
# Download SRN cars priors dataset from huggingface

TRAIN APPROPRIATE MODEL COMBINATIONS HERE

In [ ]:
!python train_network.py +dataset=cars_priors data.use_pred_depth=True opt.pretrained_channel_mismatch=True opt.lora_finetune=True
# Save weights to huggingface

In [ ]:
!python train_network.py +dataset=cars_priors data.use_pred_depth=True data.use_pred_normal=True opt.pretrained_channel_mismatch=True opt.lora_finetune=True
# Save weights to huggingface